# Dynamic Model Averaging (DMA) Replication of Koop and Korobilis (2012)
This Notebooks replicates the analysis from Koop, G. and Korobilis, D. (2012). “Forecasting Inflation Using Dynamic Model Averaging”, International Economic Review, 53, pp. 867-886.  It uses the data inlcuded in the Matlab code available at Korobilis' website (https://sites.google.com/site/dimitriskorobilis/matlab/dma).

In [29]:
%load_ext autoreload
%autoreload 2

import os
from pathlib import Path
import pickle

from src.data import import_data
from src.data.data_class import Data
from src.models.preliminaries import Settings
from src.models.dma import DMA


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Preliminaries
Specify options for the DMA. For instance, which variables to include and how to transform them.

In [26]:
params = Settings() # initialize Settings
# adjust settings
params.use_y = ['HICP_SA']  # use as y
params.use_x = ['M1', 'USD_EUR', 'OILP', 'STOCKPRICES']  # indep vars
params.tcodesX = [5, 1, 1, 1]
params.tcodey = 5
params.first_sample_ends = '1990.Q4'
params.restricted_vars = ['intercept', 'HICP_SA']
params.forgetting_method = 2
params.expert_opinion = 2
params.h_fore = 2
params.prior_theta = 1
params.miss_treatment = 2

# params.print_setting_options() # print explanation to settings
params.print_settings() # print settings

The following preliminary settings are specified:
intercept : 1
plag : 1
hlag : 0
use_x : ['M1', 'USD_EUR', 'OILP', 'STOCKPRICES']
use_y : ['HICP_SA']
tcodesX : [5, 1, 1, 1]
tcodey : 5
miss_treatment : 2
lamda : 0.99
alpha : 0.9
kappa : 0.95
forgetting_method : 2
prior_theta : 1
initial_V_0 : 2
restricted_vars : ['intercept', 'HICP_SA']
initial_DMA_weights : 1
expert_opinion : 2
h_fore : 2
first_sample_ends : 1990.Q4


## Prepare data
Load the data and transform as specified above.

In [35]:
df = import_data.load_dma_ecb() # load full dataset
df.describe()

,HICP_SA,M1,M2,M3_OUT,M3_OUTPS,LPS,D4PSTARPSCGL,D4PSTARCGL,D4DIV_TV,D4M3DEPHH,...,USD_EUR,R_EFF_EXCH,OILP,WPRM_EE,WPRM,STOCKPRICES,STOCK_PE,DIVYIELD,STOCKPRICES_EUR,UNEMPL
count,123.000000,1.220000e+02,1.220000e+02,1.220000e+02,1.220000e+02,1.220000e+02,118.000000,118.000000,75.000000,75.000000,...,123.000000,71.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,62.000000
mean,80.251870,1.881622e+06,3.837154e+06,4.375175e+06,4.284545e+06,4.838569e+06,0.039621,0.040987,6.465649,5.223127,...,1.147154,102.125070,113.248049,102.455041,95.318699,845.436098,14.566341,3.084715,711.202927,9.081129
std,18.832716,1.173273e+06,2.021400e+06,2.400648e+06,2.268884e+06,2.933337e+06,0.027796,0.027129,2.398202,2.192826,...,0.185532,7.869879,75.509877,21.052107,37.905110,602.075932,3.702359,0.930436,485.967143,1.014490
min,41.490000,5.289964e+05,1.197078e+06,1.220359e+06,1.220359e+06,1.239393e+06,-0.022522,-0.021367,1.147300,1.136172,...,0.680000,83.860000,41.870000,73.170000,50.230000,107.640000,7.170000,1.510000,93.640000,7.240000
25%,64.480000,9.487199e+05,2.192542e+06,2.331581e+06,2.331581e+06,2.314101e+06,0.017952,0.019107,4.576921,3.263218,...,1.035000,98.400000,63.585000,89.170000,63.865000,391.715000,12.640000,2.545000,352.955000,8.255000
50%,83.030000,1.434392e+06,3.351930e+06,3.896734e+06,3.896734e+06,3.958509e+06,0.036379,0.040805,6.856229,5.928227,...,1.180000,103.980000,93.300000,96.400000,84.100000,670.550000,14.020000,2.840000,525.140000,8.970000
75%,95.350000,2.599661e+06,5.004649e+06,5.859241e+06,5.639098e+06,6.770669e+06,0.055393,0.055942,8.082897,6.883072,...,1.270000,108.500000,123.365000,110.680000,111.685000,1276.395000,16.255000,3.465000,1110.555000,9.990000
max,110.530000,4.704607e+06,8.352488e+06,9.476498e+06,9.075909e+06,1.092816e+07,0.121687,0.119538,11.286172,9.043782,...,1.560000,112.950000,427.200000,182.630000,223.100000,2501.760000,25.940000,6.460000,1788.990000,10.670000


In [30]:
data = Data(df, params)
data.X.describe()

,intercept,HICP_SA,HICP_SA_t-1,M1,USD_EUR,OILP,STOCKPRICES
count,121.0,121.000000,121.000000,121.000000,121.000000,121.000000,121.000000
mean,1.0,0.031596,0.032271,0.072242,1.142975,112.923223,856.849008
std,0.0,0.022937,0.023622,0.033211,0.184150,76.093428,600.382807
min,1.0,-0.022796,-0.022796,-0.009693,0.680000,41.870000,107.640000
25%,1.0,0.017598,0.017676,0.052766,1.030000,63.470000,418.280000
50%,1.0,0.025780,0.026429,0.071277,1.170000,92.800000,680.830000
75%,1.0,0.039025,0.039061,0.095588,1.270000,120.530000,1283.400000
max,1.0,0.105961,0.105961,0.214528,1.560000,427.200000,2501.760000


## Run DMA

In [33]:
dma = DMA(params, data)
dma.run_dma()

100%|██████████| 119/119 [00:00<00:00, 542.92it/s]

DMA finished


## Analysis

In [51]:
dma.forecast_statistics()

         MAFE      MSFE      BIAS
DMA  0.001801  0.000008  0.000629
DMS  0.001666  0.000011  0.000542


In [43]:
dma.plot_inclusion_prob(seperate_plots=False, renderer='plotly_mimetype')

         MAFE      MSFE      BIAS
DMA  0.172208  0.000770  0.053089
DMS  0.149576  0.000875  0.035311


In [ ]:
dma.calc_E_size(out='plot')